In [1]:
# !nvidia-smi
import torch
from utils.utils import predict, show

## check cuda availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

/home/fame/anaconda3/envs/spanproto/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda device


In [10]:
import os
import sys
import json
import torch
import argparse
import numpy as np 

import model.model as module_arch
import utils.dataloader as module_data
from utils.metric import sequence_f1
from utils.parse_config import ConfigParser

args = argparse.ArgumentParser(description='PyTorch Template')
args.add_argument('-c', '--config', default=None, type=str, help='config file path (default: None)')
args.add_argument('-d', '--device', default=device, type=str, help='indices of GPUs to enable (default: all)')
args.add_argument('-r', '--resume', default='storage/checkpoints/debug_model/0806_234452_318276-based-lst20-lr_0.001-bs_16/model_best.pth', type=str, help='path to latest checkpoint (default: None)')
args.add_argument('--f', default='file', type=None)

# Check if in IPython environment
if any("ipykernel" in arg for arg in sys.argv):
    sys.argv = sys.argv[:1]  # Keep only the script name
    
config = ConfigParser.from_args(args)
logger = config.get_logger('test')

# setup dataloader instances
data_loader = config.init_obj('dataloader', module_data)

# build model architecturea
model = config.init_obj('arch', module_arch, num_tag=data_loader.num_tag, path_lm=data_loader.path_lm)

# get function handles of loss and metrics
metric_fns = {"sequence_f1": sequence_f1}

## Load checkpoint
logger.info('Loading checkpoint: {} ...'.format(config.resume))
checkpoint = torch.load(config.resume)
state_dict = checkpoint['state_dict']
model.load_state_dict(state_dict)
model.eval()
model = model.to(device)

## 
text = """
เมื่อวันที่29ก.ค.55 สำนักข่าวบีบีซีรายงานว่า มหาวิทยาลัยออกซ์ฟอร์ดของอังกฤษได้แก้ไขกฎการแต่งกายภายในมหาวิทยาลัย 
หลังชมรมเพื่อความหลากหลายทางเพศยื่นคำร้องว่ากฎที่มีอยู่เดิมไม่เป็นธรรมกับกลุ่มนศข้ามเพศtransgenderใน
มหาวิทยาลัย
"""
# Setup
tokens, out = predict(
    model, text,
    data_loader.path_lm, 
    data_loader.ids2spantag, 
    data_loader.sent_length
)

tokens = [tk for tk in tokens if tk!=data_loader.pad]
print("|".join(tokens), "\n")
[show(x) for x in out];

Train : 10 sentences
Dev : 10 sentences
Test : 7 sentences
Max sents length: 256 tokens
num_vocab: 25004
num_tag: 12
num_span: 5
num_spantag: 45


Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:test:Loading checkpoint: storage/checkpoints/debug_model/0806_234452_318276-based-lst20-lr_0.001-bs_16/model_best.pth ...


<s>||เมื่อ|วันที่||29||ก|.|ค|.||55|||สํานักข่าว||บีบีซี||รายงาน||ว่า|||มหาวิทยาลั|ย||ออก|ซ์||ฟอร์ด||ของ||อังกฤษ||ได้||แก้ไข||กฎ|การ||แต่งกาย||ภายใน||มหาวิทยาลั|ย|||หลัง||ชมรม|เพื่อ|ความ|หลากหลาย||ทางเพศ||ย|ื่นคําร้อง||ว่า||กฎ||ที่|มี||อยู่||เดิม|ไม่||เป็นธรรม||กับ|กลุ่ม||นศ||ข้าม||เพศ||trans|g|ender||ใน|||มหาวิทยาลั|ย||</s> 

